<a href="https://colab.research.google.com/github/adidam/rag-impl/blob/main/rag_tokenizer_chromadb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision datasets transformers pymilvus pymilvus[model]
!pip install groq
!pip install -q langchain langchain-groq


In [ ]:
import numpy as np
import os
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import random_split
import torch.optim as optim
import matplotlib.pyplot as plt
import logging
from datasets import load_dataset
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer
import nltk
from datasets import load_dataset
#import chromadb
#from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import time

nltk.download('punkt_tab')

GROQ_KEY = "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" #os.environ.get('GROQ_API_KEY')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
import numpy as np
from pymilvus import connections
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
from pymilvus import MilvusClient
from pymilvus import utility

class VectorDataStore:
    db_url = "http://localhost:19530"
    #description = f"collection created for {self.name}"

    def __init__(self, path="/content/ragbench.db"):
        self.client = MilvusClient(path)



    def create_collection(self, name, vec_dim=128):
        if self.client.has_collection(name):
            self.default_collection = name

        self.description = f"collection to store {name}"

        index_params = self.client.prepare_index_params()
        index_params.add_index(
            field_name="embedding",
            index_type="AUTOINDEX",
            metric_type="COSINE"
        )
        schema = self.client.create_schema(
            auto_id=False,
            enable_dynamic_fields=True,
        )
        schema.add_field(field_name="pk", datatype=DataType.VARCHAR, max_length=64, is_primary=True)
        schema.add_field(field_name="metadata", datatype=DataType.JSON)
        schema.add_field(field_name="documents", datatype=DataType.VARCHAR, max_length=512)
        schema.add_field(field_name="embedding", datatype=DataType.FLOAT_VECTOR, dim=vec_dim)

        collection = self.client.create_collection(collection_name=name,
                                       schema=schema,
                                       index_params=index_params)
        return collection


    def get_collection(self, name):
        if not self.client.has_collection(name):
            raise ValueError(f"Collection '{name}' does not exist.")
        self.current_collection = Collection(name)
        return self.current_collection.num_entities

    def insert(self, collection_name: str, metadata: list[dict[str, any]],
                documents: list[str], embeddings: np.ndarray, ids: list[int]):

        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist. Create it first.")

        if len(metadata) != len(embeddings) != len(documents) != len(ids):
           raise ValueError("Metadata, documnets, ids and embeddings must have the same length.")

        data = []
        for meta, doc, emb, id in zip(metadata, documents, embeddings, ids):
          datum = {
              "pk": id,
              "metadata": meta,
              "documents": doc,
              "embedding": emb.tolist(),
          }
          data.append(datum)

        self.client.insert(collection_name, data)
        print(f"Inserted {len(metadata)} records into collection '{collection_name}'.")

    def drop_collection(self, collection_name: str):
        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist.")
        self.client.drop_collection(collection_name)
        print(f"Dropped collection '{collection_name}'.")

    def delete_all(self, collection_name: str):
        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist.")
        self.client.delete_collection(collection_name)

    def search(self, query_embedding: np.ndarray, top_k: int = 10) -> list[dict[str, any]]:
        """
        Search across all collections for the top-k closest embeddings.
        :param query_embedding: The embedding vector to search for.
        :param top_k: Number of top results to retrieve.
        :return: A list of dictionaries containing collection name, id, metadata, and distance.
        """
        results = []
        collections = self.client.list_collections()
        start_time = time.time()
        for collection_name in collections:
            if not self.client.has_collection(collection_name):
                continue

            # Set params to COSINE to match chromadb
            search_params = {"metric_type": "COSINE", "params": {"ef": 128}}

            search_results = self.client.search(
                collection_name=collection_name,
                data=[query_embedding.tolist()],
                anns_field="embedding",
                search_params=search_params,
                limit=top_k,
                output_fields=["metadata", "documents"]
            )

            for hits in search_results:
                for hit in hits:
                    print(f"Collection: {collection_name}, data: {str(hit)}")
                    results.append({
                        "collection": collection_name,
                        "id": hit["id"],
                        "metadata": hit["entity"]["metadata"],
                        "distance": hit["distance"],
                        "documents": hit["entity"]["documents"]
                      })

        results = sorted(results, key=lambda x: x["distance"])[:top_k]
        end_time = time.time()
        print(f"Search completed. Found {len(results)} results. in {end_time - start_time} secs")
        return results

    def extract_documents(self, search_results: list[dict[str, any]]) -> list[np.ndarray]:
      """
      Extract embedding values from search results.
      :param search_results: List of dictionaries containing search results.
      :return: List of embedding vectors as NumPy arrays.
      """
      return [np.array(result["documents"]) for result in search_results if "documents" in result]


In [ ]:
def chunk_with_token_limit(text, token_limit, overlap):
    sentences = sent_tokenize(text)  # Split text into sentences
    chunks = []  # Store resulting chunks
    current_chunk = []  # Temporarily hold sentences for the current chunk
    current_chunk_tokens = 0  # Token count for the current chunk

    for sentence in sentences:
        # Tokenize the sentence and calculate its token count
        sentence_tokens = tokenizer.tokenize(sentence)
        num_tokens = len(sentence_tokens)

        # print(f"Tokens: {sentence_tokens[0]}")

        # If adding this sentence exceeds the token limit
        if current_chunk_tokens + num_tokens > token_limit:
            # Save the current chunk
            chunk_text = " ".join(current_chunk)
            chunks.append(chunk_text)

            # Prepare the next chunk with overlap
            overlap_tokens = tokenizer.tokenize(" ".join(current_chunk[-1:]))
            current_chunk = [sentence for sentence in current_chunk[-(overlap // len(overlap_tokens)) :]] if current_chunk else []
            current_chunk_tokens = sum(len(tokenizer.tokenize(sent)) for sent in current_chunk)

        # Add the sentence to the current chunk
        current_chunk.append(sentence)
        current_chunk_tokens += num_tokens

    # Add the last chunk if it exists
    if current_chunk:
        chunk_text = " ".join(current_chunk)
        chunks.append(chunk_text)

    return chunks

# Function for processing the document and adding identifiers
def process_document_with_identifiers(document):
    processed_data = []
    title_count = 0

    for section in document:
        section_chunks = []
        passage_count = ord("a")  # Initialize passage letter as 'a'

        sentences = sent_tokenize(section)
        for sentence in sentences:
            if sentence.startswith("Title:"):
                identifier = f"{title_count}a"
                chunked_texts = chunk_with_token_limit(sentence, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)
                for chunk in chunked_texts:
                    section_chunks.append([identifier, chunk])
                title_count += 1
                passage_count = ord("a")  # Reset passage count for the next title
            elif sentence.startswith("Passage:"):
                identifier = f"{title_count - 1}{chr(passage_count)}"
                chunked_texts = chunk_with_token_limit(sentence, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)
                for chunk in chunked_texts:
                    section_chunks.append([identifier, chunk])
                passage_count += 1
            else:
                identifier = f"{title_count - 1}{chr(passage_count)}"
                chunked_texts = chunk_with_token_limit(sentence, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)
                for chunk in chunked_texts:
                    section_chunks.append([identifier, chunk])
                passage_count += 1

        processed_data.append(section_chunks)
    return processed_data

In [ ]:
# Function to retrieve relevant chunks
def retrieve_docs_milvus(query, encoder, top_k=5):
    # Generate embedding for the query
    query_embedding = encoder.encode(query)
    # Perform vector search to find relevant chunks
    results = datastor.extract_documents(datastor.search(query_embedding, top_k))
    print(f"results: {results}")
    # Extract the retrieved chunks
    chunks = results
    # should sort and push context - but later
    return chunks

In [ ]:
def query_response_from_llm(query: str, encoder):
    # chunks = retrieve_docs(query)
    chunks = retrieve_docs_milvus(query, encoder)
    # Flatten the list if necessary
    if any(isinstance(chunk, list) for chunk in chunks):
      chunks = [item for sublist in chunks for item in (sublist if isinstance(sublist, list) else [sublist])]

    chat = ChatGroq(temperature=0.3, groq_api_key=GROQ_KEY, model_name="llama3-8b-8192")

    prompt=ChatPromptTemplate.from_template(
      """
      Please provide a response to the query below, strictly adhering to the
      information presented in the following documents.
      Do not generate any text beyond what is explicitly stated in the documents.

      Context: {context}

      Question: {query}

      Answer:
      """
    )

    chain = prompt | chat
    context = chunks
    groq_response = chain.invoke({"context": context, "query": query})

    print("groq_response>>>RAG>>>>>> ",groq_response)

    answer = groq_response
    return answer, context

def query_response_from_llm_no_rag(query: str):

    chat = ChatGroq(temperature=0.3, groq_api_key=GROQ_KEY, model_name="llama3-8b-8192")

    prompt=ChatPromptTemplate.from_template(
      """
      Please provide a response to the query below

      Question: {query}

      Answer:
      """
    )

    chain = prompt | chat

    groq_response_no_rag = chain.invoke({"query": query})

    print("groq_response_no_rag>>>no RAG>>>>>> ",groq_response_no_rag)

    answer = groq_response_no_rag
    return answer

def generate_prompt():
    """
    Generate a prompt template for assessing the support and relevance of an LLM-generated response.
    """
    return """
    I asked someone to answer a question based on one or more documents.
    Your task is to review their response and assess whether or not each sentence
    in that response is supported by text in the documents. And if so, which
    sentences in the documents provide that support. You will also tell me which
    of the documents contain useful information for answering the question, and
    which of the documents the answer was sourced from.
    Here are the documents, each of which is split into sentences.Alongside each
    sentence is associated key, such as ’[0a].’ or ’[0b].’ that you can use to refer
    to it:

    ‘‘‘
    {documents}
    ‘‘‘
    The question was:
    ‘‘‘
    {question}
    ‘‘‘

    Here is their response, split into sentences. Alongside each sentence is
    associated key, such as ’a.’ or ’b.’ that you can use to refer to it. Note
    that these keys are unique to the response, and are not related to the keys
    in the documents:
    ‘‘‘
    {answer}
    ‘‘‘
    You must respond with a JSON object matching this schema:
    ‘‘‘
    {{
    "relevance_explanation": string,
    "all_relevant_sentence_keys": [string],
    "overall_supported_explanation": string,
    "overall_supported": boolean,
    "sentence_support_information": [
    {{
    "response_sentence_key": string,
    "explanation": string,
    "supporting_sentence_keys": [string],
    "fully_supported": boolean
    }},
    ],
    "all_utilized_sentence_keys": [string]
    }}
    ‘‘‘
    The relevance_explanation field is a string explaining which documents
    contain useful information for answering the question. Provide a step-by-step
    breakdown of information provided in the documents and how it is useful for
    answering the question.
    The all_relevant_sentence_keys field is a list of all document sentences keys
    (e.g. ’0a’) that are relevant to the question. Include every sentence that is
    useful and relevant to the question, even if it was not used in the response,
    or if only parts of the sentence are useful. Ignore the provided response when
    making this judgement and base your judgement solely on the provided documents
    and question. Omit sentences that, if removed from the document, would not
    impact someone’s ability to answer the question.
    The overall_supported_explanation field is a string explaining why the response
    *as a whole* is or is not supported by the documents. In this field, provide a
    step-by-step breakdown of the claims made in the response and the support (or
    lack thereof) for those claims in the documents. Begin by assessing each claim
    separately, one by one; don’t make any remarks about the response as a whole
    until you have assessed all the claims in isolation.
    The overall_supported field is a boolean indicating whether the response as a
    whole is supported by the documents. This value should reflect the conclusion
    you drew at the end of your step-by-step breakdown in overall_supported_explanation.
    In the sentence_support_information field, provide information about the support
    *for each sentence* in the response.
    The sentence_support_information field is a list of objects, one for each sentence
    in the response. Each object MUST have the following fields:
    - response_sentence_key: a string identifying the sentence in the response.
    This key is the same as the one used in the response above.

    - explanation: a string explaining why the sentence is or is not supported by the
    documents.
    - supporting_sentence_keys: keys (e.g. ’[0a]’) of sentences from the documents that
    support the response sentence. If the sentence is not supported, this list MUST
    be empty. If the sentence is supported, this list MUST contain one or more keys.
    In special cases where the sentence is supported, but not by any specific sentence,
    you can use the string "supported_without_sentence" to indicate that the sentence
    is generally supported by the documents. Consider cases where the sentence is
    expressing inability to answer the question due to lack of relevant information in
    the provided context as "supported_without_sentence". In cases where the sentence
    is making a general statement (e.g. outlining the steps to produce an answer, or
    summarizing previously stated sentences, or a transition sentence), use the
    string "general". In cases where the sentence is correctly stating a well-known fact,
    like a mathematical formula, use the string "well_known_fact". In cases where the
    sentence is performing numerical reasoning (e.g. addition, multiplication), use
    the string "numerical_reasoning".
    - fully_supported: a boolean indicating whether the sentence is fully supported by
    the documents.
    - This value should reflect the conclusion you drew at the end of your step-by-step
    breakdown in explanation.
    - If supporting_sentence_keys is an empty list, then fully_supported must be false.
    - Otherwise, use fully_supported to clarify whether everything in the response
    sentence is fully supported by the document text indicated in supporting_sentence_keys
    (fully_supported = true), or whether the sentence is only partially or incompletely
    supported by that document text (fully_supported = false).
    The all_utilized_sentence_keys field is a list of all sentences keys (e.g. ’0a’) that
    were used to construct the answer. Include every sentence that either directly supported
    the answer, or was implicitly used to construct the answer, even if it was not used
    in its entirety. Omit sentences that were not used, and could have been removed from
    the documents without affecting the answer.
    You must respond with a valid JSON string. Use escapes for quotes, e.g. \\"\\", and
    newlines, e.g. \\n. Do not write anything before or after the JSON string. Do not
    wrap the JSON string in backticks like ‘‘‘ or ‘‘‘json.
    As a reminder: your task is to review the response and assess which documents contain
    useful information pertaining to the question, and how each sentence in the response
    is supported by the text in the documents.
    """.strip()


In [ ]:
logger = logging.getLogger(__name__)

#model_name = "BAAI/LLM-Embedder"
model_name = "sentence-transformers/all-MiniLM-L6-v2"

# load the full ragbench dataset
ragbench = {}
# for dataset in ['covidqa', 'cuad', 'delucionqa', 'emanual', 'expertqa', 'finqa', 'hagrid', 'hotpotqa', 'msmarco', 'pubmedqa', 'tatqa', 'techqa']:
# for dataset in ['hagrid', 'hotpotqa', 'msmarco']:
#   ragbench[dataset] = load_dataset("rungalileo/ragbench", dataset)

# len(ragbench['hotpotqa']['train'])

#tokenizer = AutoTokenizer.from_pretrained("BAAI/LLM-Embedder")
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Sliding window configuration
TOKEN_LIMIT = 256
SLIDING_WINDOW_OVERLAP = 100  # Overlap between consecutive chunks (in tokens)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 9919aa01af494ae5bd583f027b6c385b


In [ ]:
datasets = ['hagrid', 'hotpotqa', 'msmarco'] # ["covidqa"]

datastor = VectorDataStore()

collection_name = "ragbench_hhm_v1"

# Step 4: Generate Embeddings
model = SentenceTransformer(model_name)  # Pretrained sentence transformer
batch_size = 2500  # Adjust based on available memory

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

model.to(device)

num_records = 0
if datastor.client.has_collection(collection_name):
  num_records = datastor.get_collection(collection_name)
else:
  datastor.create_collection(datasets[0], model.get_sentence_embedding_dimension())

if num_records == 0:
  all_documents = []
  all_ids = []
  all_metadatas = []

  # Process each dataset
  doc_idx = 0  # Global document index for unique IDs
  for dataset in datasets:
      data = load_dataset("rungalileo/ragbench", dataset, split="train")

      for idx, row in tqdm(enumerate(data), desc=f"Processing {dataset}"):
          # Extract document text
          doc_text = row.get('documents', '')

          # Skip if no documents found
          if not doc_text:
              continue

          # Process the document
          processed_output = process_document_with_identifiers(doc_text)

          # Populate the lists
          for section_idx, section in enumerate(processed_output):
              for item_idx, (prefix, content) in enumerate(section):
                  # Add the document
                  document = f"[{prefix}] {content}"
                  all_documents.append(document)

                  # Construct a globally unique ID
                  doc_id = f"{dataset}_{doc_idx}_{section_idx}_{item_idx}"
                  all_ids.append(doc_id)

                  # Construct metadata
                  metadata = {
                      "dataset": dataset,
                      "global_index": doc_idx,
                      "section_index": section_idx,
                      "item_index": item_idx,
                      "prefix": prefix,
                      "type": "Title" if prefix.endswith("a") else "Passage",
                  }
                  all_metadatas.append(metadata)

          doc_idx += 1  # Increment global document index

  # Generate embeddings in batches
  all_embeddings = []
  for i in tqdm(range(0, len(all_documents), batch_size), desc="Generating embeddings"):
      batch_docs = all_documents[i:i + batch_size]
      batch_embeddings = model.encode(batch_docs, show_progress_bar=True)
      all_embeddings.extend(batch_embeddings)

  # insert data into the collection
  for i in tqdm(range(0, len(all_documents), batch_size), desc="Adding data to Milvus"):
    batch_embeddings = all_embeddings[i:i + batch_size]
    batch_metadatas = all_metadatas[i:i + batch_size]
    batch_documents = all_documents[i:i + batch_size]
    batch_ids = all_ids[i:i + batch_size]

    # Add the batch to the ChromaDB collection
    datastor.insert(datasets[0],
        metadata=batch_metadatas,
        documents=batch_documents,
        embeddings=np.array(batch_embeddings),
        ids=batch_ids
    )


DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 05a7e4ef0cd945d0b820e7109036a047
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: hagrid


Using device: cuda


DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: hagrid


train-00000-of-00001.parquet:   0%|          | 0.00/9.42M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

...

Inserted 2500 records into collection 'hagrid'.


Adding data to Milvus:  98%|█████████▊| 54/55 [01:10<00:01,  1.21s/it]

Inserted 2500 records into collection 'hagrid'.


Adding data to Milvus: 100%|██████████| 55/55 [01:11<00:00,  1.30s/it]

Inserted 2320 records into collection 'hagrid'.


In [ ]:
from pymilvus import __version__
print(f"Milvus Python SDK version: {__version__}")

ques = "When was Rolex founded?"

vec = model.encode(ques)

print(len(vec))

res = datastor.search(vec, 10)

for r in res:
  print(r)

res_vec = datastor.extract_documents(res)



Milvus Python SDK version: 2.5.0
384
Collection: hagrid, data: {'id': 'hagrid_0_0_1', 'distance': 0.6551020741462708, 'entity': {'documents': '[-1b] Originally founded as "Wilsdorf and Davis" by Hans Wilsdorf and Alfred Davis in London in 1905, the company registered "Rolex" as the brand name of its watches in 1908.', 'metadata': {'dataset': 'hagrid', 'global_index': 0, 'section_index': 0, 'item_index': 1, 'prefix': '-1b', 'type': 'Passage'}}}
Collection: hagrid, data: {'id': 'hagrid_0_0_2', 'distance': 0.5708330273628235, 'entity': {'documents': '[-1c] In 1919, the company moved its base of operations to Geneva, Switzerland in order to avoid heavy taxation from a recovering post-war Britain, and "Montres Rolex S.A." was registered as the new company name by Hans Wilsdorf in Geneva in 1920.', 'metadata': {'dataset': 'hagrid', 'global_index': 0, 'section_index': 0, 'item_index': 2, 'prefix': '-1c', 'type': 'Passage'}}}
Collection: hagrid, data: {'id': 'hagrid_0_0_0', 'distance': 0.54146

In [ ]:
print(f"records: {len(res_vec)}")
for r in res_vec:
  print(f"result documents: {r}")

records: 10
result documents: [-1c] Dumont founded the "Groupe des XXX" (1907), and along with Robert Antoine Pinchon, Yvonne Barbier, and Eugène Tirvert founded the Société Normande de Peinture Moderne (1909).
result documents: [-1b] The firm was founded in May 2003 by veteran U.S. Army Special Forces Soldiers, including former Delta Operators.
result documents: [-1b] It was founded in 1891 by Anthony J. Drexel, a noted financier and philanthropist.
result documents: [-1b] It was founded in 1891 by Anthony J. Drexel, a noted financier and philanthropist.
result documents: [-1e] Since then GeneDx has operated as a subsidiary of this parent company under the leadership of Drs.
result documents: [-1c] Founded in 1986 by David J. Theroux, the Institute focuses on political, social, economic, legal, environmental and foreign policy issues.
result documents: [-1c] The company also operates one Rolex-branded store in Orlando.
result documents: [-1a] Rolex SA () is a Swiss luxury watchmaker.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [ ]:
(groq_response, context) = query_response_from_llm(ques, model)
print("answer  >>after>>> ", groq_response)

Collection: hagrid, data: {'id': 'hagrid_0_0_1', 'distance': 0.6551020741462708, 'entity': {'documents': '[-1b] Originally founded as "Wilsdorf and Davis" by Hans Wilsdorf and Alfred Davis in London in 1905, the company registered "Rolex" as the brand name of its watches in 1908.', 'metadata': {'dataset': 'hagrid', 'global_index': 0, 'section_index': 0, 'item_index': 1, 'prefix': '-1b', 'type': 'Passage'}}}
Collection: hagrid, data: {'id': 'hagrid_0_0_2', 'distance': 0.5708330273628235, 'entity': {'documents': '[-1c] In 1919, the company moved its base of operations to Geneva, Switzerland in order to avoid heavy taxation from a recovering post-war Britain, and "Montres Rolex S.A." was registered as the new company name by Hans Wilsdorf in Geneva in 1920.', 'metadata': {'dataset': 'hagrid', 'global_index': 0, 'section_index': 0, 'item_index': 2, 'prefix': '-1c', 'type': 'Passage'}}}
Collection: hagrid, data: {'id': 'hagrid_0_0_0', 'distance': 0.5414658784866333, 'entity': {'documents': 

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chat = ChatGroq(temperature=0.3, groq_api_key=GROQ_KEY, model_name="llama3-8b-8192")

prompt_template_with_docs = PromptTemplate(
    input_variables=["documents", "question", "answer"],
    template=generate_prompt(),
)

print('context for groq >>>> ', context)
print('query for groq >>>> ', ques)
print('answer for groq >>>> ', groq_response)

chain = prompt_template_with_docs | chat

# if classification == 1:
#  groq_response_with_context_qanda = chain.invoke({"documents": context, "question": query, "answer":groq_response})
# else
#  groq_response_with_out-rag = chain.invoke({"documents": "", "question": query, "answer":groq_response})

groq_response_with_context_qanda = chain.invoke({"documents": context, "question": ques, "answer":groq_response})

print("groq_response>>>>with context, query and answer>>>>> ",groq_response_with_context_qanda)

context for groq >>>>  [array('[-1c] Founded in 1986 by David J. Theroux, the Institute focuses on political, social, economic, legal, environmental and foreign policy issues.',
      dtype='<U144'), array('[-1c] The company also operates one Rolex-branded store in Orlando.',
      dtype='<U67'), array('[-1a] Rolex SA () is a Swiss luxury watchmaker.', dtype='<U47'), array('[-1c] In 1919, the company moved its base of operations to Geneva, Switzerland in order to avoid heavy taxation from a recovering post-war Britain, and "Montres Rolex S.A." was registered as the new company name by Hans Wilsdorf in Geneva in 1920.',
      dtype='<U247'), array('[-1b] Originally founded as "Wilsdorf and Davis" by Hans Wilsdorf and Alfred Davis in London in 1905, the company registered "Rolex" as the brand name of its watches in 1908.',
      dtype='<U174')]
query for groq >>>>  When was Rolex founded?
answer for groq >>>>  content='According to the documents, Rolex was founded in 1905 as "Wilsdorf an